In [ ]:
# I was getting a ModuleNotFoundError, and I discovered that the cwd (current working directory) was
#   '<Github Home>/market-modeling/examples/' but it needs to be '<Github Home>/market-modeling/'
# The following code will add the cwd's parent directory to the path Python uses to search for packages
import os
import sys

sys.path.append(os.path.dirname(os.getcwd()))

In [ ]:
import os
from datetime import datetime

from gbm_models.Models import DependentModel
from markets.Markets import XarrayMarket
from assets.Assets import BaseAsset
from metrics.Metrics import VarType, TimeseriesMetric
from py_utils.py_utils import last_index_lt_1D

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
import xarray as xr

# for auto-reloading extenrnal modules
%load_ext autoreload
%autoreload 2

In [ ]:
DEBUG = False
DATA_DIR = '../data'
MARKET_NAME = 'popular_stocks'
TS_DIR = 'growth_timeseries'

In [ ]:
asset_list = []
asset_metadata = pd.read_csv(os.path.join(DATA_DIR, MARKET_NAME, 'asset_metadata.tsv'), sep='\t')

for i, row in asset_metadata.iterrows():
  if DEBUG:
    if i == 5:
      break
  
  asset_ts = pd.read_csv(
    os.path.join(DATA_DIR, MARKET_NAME, TS_DIR, f"{row['id']}.csv"),
    parse_dates=['date'],
    dtype={'price' : np.float32}
  )
  
  asset = BaseAsset(
    row['name'],
    row['id'],
    {
      'signal': TimeseriesMetric(asset_ts['date'], asset_ts['price'], VarType.QUANTITATIVE)
    }
  )
  asset_list.append(asset)
  
market = XarrayMarket(asset_list, MARKET_NAME)
dep_model = DependentModel(market)

market.align_timeseries()

In [ ]:
market.display_market()

In [ ]:
dep_model.estimate_parameters()

In [ ]:
dep_model.drifts

In [ ]:
Sigma = dep_model.get_correlation_mat()

plt.set_cmap('bwr')
cust_cmap = TwoSlopeNorm(0, Sigma.min(), Sigma.max())
plt.matshow(Sigma, norm=cust_cmap)
plt.xticks(np.arange(Sigma.shape[0]), labels=[asset.asset_id for asset in market.assets], rotation=60)
plt.yticks(np.arange(Sigma.shape[1]), labels=[asset.asset_id for asset in market.assets])
plt.colorbar();

In [ ]:
sim_dates = pd.to_datetime(
  pd.Series(['01-24-2025', '01-27-2025', '01-28-2025', '01-29-2025'], name='dates'),
  format='%m-%d-%Y'
)

sim_res = dep_model.simulate(sim_dates, num_sims=20)
sim_counterfact = dep_model.simulate(sim_dates, 1, add_BM=False)

In [ ]:
colors = ['C0', 'C1', 'C2', 'C3', 'C4']
fig, ax = plt.subplots(figsize=(10,8))

for i in range(len(colors)): #range(dep_model.N_ASSETS):
  asset_i = dep_model.market.assets[i % dep_model.N_ASSETS]

  recent_idx = last_index_lt_1D(market.xarray.time.values, np.datetime64(datetime(2024, 1, 1)))

  disp_name = asset_i.asset_id
  ax.plot(
    market.xarray.isel(time=slice(recent_idx, None)).time, market.xarray.isel(time=slice(recent_idx, None)).sel(ID=asset_i.asset_id, variable='signal'),
    '-o', color=colors[i % dep_model.N_ASSETS],
    label=f'{disp_name} true signal'
  )
  
  fut_dates = pd.concat((market.xarray.time[-1:].to_pandas(), sim_dates))
  # ax.set_xlim([datetime(2025, 1, 1), sim_dates.max()])
  ax.plot(
    fut_dates, sim_counterfact[0, i, :],
    color=colors[i % dep_model.N_ASSETS], marker='.', linestyle='dashed',
    label=f'{disp_name} avg prediction'
  )
  for j_sim in range(sim_res.shape[0]):
    ax.plot(
      fut_dates, sim_res[j_sim, i, :],
      alpha=0.5, color=colors[i % dep_model.N_ASSETS], linestyle='dotted', marker='.'
    )
ax.set(xlabel='Date', ylabel='close price', title='Close Prices vs Time')
ax.set_yscale('log')
fig.tight_layout()
fig.legend(loc=2);